# Day 11 "Dumbo Octopus"

## Part 1

### Problem

You enter a large cavern full of rare bioluminescent dumbo octopuses! They seem to not like the Christmas lights on your submarine, so you turn them off for now.

There are 100 octopuses arranged neatly in a 10 by 10 grid. Each octopus slowly gains energy over time and flashes brightly for a moment when its energy is full. Although your lights are off, maybe you could navigate through the cave without disturbing the octopuses if you could predict when the flashes of light will happen.

Each octopus has an energy level - your submarine can remotely measure the energy level of each octopus (your puzzle input). For example:

    5483143223
    2745854711
    5264556173
    6141336146
    6357385478
    4167524645
    2176841721
    6882881134
    4846848554
    5283751526

The energy level of each octopus is a value between 0 and 9. Here, the top-left octopus has an energy level of 5, the bottom-right one has an energy level of 6, and so on.

You can model the energy levels and flashes of light in steps. During a single step, the following occurs:

* First, the energy level of each octopus increases by 1.
* Then, any octopus with an energy level greater than 9 flashes. This increases the energy level of all adjacent octopuses by 1, including octopuses that are diagonally adjacent. If this causes an octopus to have an energy level greater than 9, it also flashes. This process continues as long as new octopuses keep having their energy level increased beyond 9. (An octopus can only flash at most once per step.)
* Finally, any octopus that flashed during this step has its energy level set to 0, as it used all of its energy to flash.

Adjacent flashes can cause an octopus to flash on a step even if it begins that step with very little energy. Consider the middle octopus with 1 energy in this situation:

Before any steps:

    11111
    19991
    19191
    19991
    11111

After step 1:

    34543
    40004
    50005
    40004
    34543

After step 2:

    45654
    51115
    61116
    51115
    45654

An octopus is highlighted when it flashed during the given step.

Here is how the larger example above progresses:

Before any steps:

    5483143223
    2745854711
    5264556173
    6141336146
    6357385478
    4167524645
    2176841721
    6882881134
    4846848554
    5283751526

After steps 1-5:

    6594254334    8807476555    0050900866    2263031977    4484144000
    3856965822    5089087054    8500800575    0923031697    2044144000
    6375667284    8597889608    9900000039    0032221150    2253333493
    7252447257    8485769600    9700000041    0041111163    1152333274
    7468496589    8700908800    9935080063    0076191174    1187303285
    5278635756    6600088989    7712300000    0053411122    1164633233
    3287952832    6800005943    7911250009    0042361120    1153472231
    7993992245    0000007456    2211130000    5532241122    6643352233
    5957959665    9000000876    0421125000    1532247211    2643358322
    6394862637    8700006848    0021119000    1132230211    2243341322

After steps 6-10:

    5595255111    6707366222    7818477333    9060000644    0481112976
    3155255222    4377366333    5488477444    7800000976    0031112009
    3364444605    4475555827    5697666949    6900000080    0041112504
    2263444496    3496655709    4608766830    5840000082    0081111406
    2298414396    3500625609    4734946730    5858000093    0099111306
    2275744344    3509955566    4740097688    6962400000    0093511233
    2264583342    3486694453    6900007564    8021250009    0442361130
    7754463344    8865585555    0000009666    2221130009    5532252350
    3754469433    4865580644    8000004755    9111128097    0532250600
    3354452433    4465574644    6800007755    7911119976    0032240000

After step 10, there have been a total of 204 flashes. Fast forwarding, here is the same configuration every 10 steps:

After steps 20, 30, 40, 50, 60:

    3936556452    0643334118    6211111981    9655556447    2533334200
    5686556806    4253334611    0421111119    4865556805    2743334640
    4496555690    3374333458    0042111115    4486555690    2264333458
    4448655580    2225333337    0003111115    4458655580    2225333337
    4456865570    2229333338    0003111116    4574865570    2225333338
    5680086577    2276733333    0065611111    5700086566    2287833333
    7000009896    2754574565    0532351111    6000009887    3854573455
    0000000344    5544458511    3322234597    8000000533    1854458611
    6000000364    9444447111    2222222976    6800000633    1175447111
    4600009543    7944446119    2222222762    5680000538    1115446111

After steps 70, 80, 90, 100:

    8211111164    1755555697    7433333522    0397666866
    0421111166    5965555609    2643333522    0749766918
    0042111114    4486555680    2264333458    0053976933
    0004211115    4458655580    2226433337    0004297822
    0000211116    4570865570    2222433338    0004229892
    0065611111    5700086566    2287833333    0053222877
    0532351111    7000008666    2854573333    0532222966
    7322235117    0000000990    4854458333    9322228966
    5722223475    0000000800    3387779333    7922286866
    4572222754    0000000000    3333333333    6789998766

After 100 steps, there have been a total of 1656 flashes.

Given the starting energy levels of the dumbo octopuses in your cavern, simulate 100 steps. How many total flashes are there after 100 steps?

### Setup

Data model is a simple array, with a `-1` border to simplify the code by getting rid of bounds tests, which also enables nice offset-based addressing.

In [211]:
from utils import *

_input = initDay("day11")

_sample1 = '''
    11111
    19991
    19191
    19991
    11111
    '''

_sample2 = '''
    5483143223
    2745854711
    5264556173
    6141336146
    6357385478
    4167524645
    2176841721
    6882881134
    4846848554
    5283751526
    '''

class Board:
    def __init__(self, text):
        lines = [l.strip() for l in text.strip().splitlines()]  # parse
        self.cx, self.cy = len(lines[0])+2, len(lines)+2        # dimensions w border

        self.grid = flatten(
            [-1]*self.cx +                                      # top border
            [[-1]+[int(c) for c in l]+[-1] for l in lines] +    # line with borders
            [-1]*self.cx)                                       # bottom border

        self.adj = [                            # adjacent cell offsets
            -self.cx-1, -self.cx, -self.cx+1,   # prev row
            -1, 1,                              # same row
            self.cx-1, self.cx, self.cx+1]      # next row

    def dump(self):
        for y in range(1, self.cy-1):
            for x in range(self.cx):
                match self.grid[self.cx*y+x]:
                    case 10:
                        print('*', end='')
                    case _ as i if i >= 0:
                        print(i, end='')
            print()
        print()

### Solution

Straightforward problem that I initially screwed up by not properly marking cells as visited (even though I thought I was doing so). It works like this:

1. Increment every cell
2. Flash every 10 cell by recursively visiting the 8 adjacents, incrementing and potentially flashing each. Mark flashed cells as 11 so we don't hit them again.
3. Reset 10's and 11's to 0 and also include those in the final score we return.

It turns out the simulation doesn't change between parts, so have a `sim` function that just runs a single step and let the caller decide how to score things or when to end.

In [212]:
def sim(b):
    b.grid = [c + (1 if c>=0 else 0) for c in b.grid] # increment all except border

    def flash(base):
        b.grid[base] = 11
        for o in [base+v for v in b.adj]:
            c = b.grid[o]
            if c>=0 and c<10:
                b.grid[o] += 1
                if c == 9:
                    flash(o)

    for o, v in enumerate(b.grid): # flash the fresh 10's
        if v == 10:
            flash(o)

    b.grid = [(0 if c>=10 else c) for c in b.grid] # pop 10's and 11's back to 0

def solve1(text, steps):
    b, result = Board(text), 0
    for _ in range(steps):
        sim(b)
        result += sum([1 for c in b.grid if not c])
    return result

check(solve1(_sample1, 2), 9)
check(solve1(_sample2, 10), 204)
check(solve1(_sample2, 100), 1656)
check1(solve1(_input, 100))

Part 1 Result: 1627


## Part 2

### Problem

It seems like the individual flashes aren't bright enough to navigate. However, you might have a better option: the flashes seem to be synchronizing!

In the example above, the first time all octopuses flash simultaneously is step 195:

After step 193:

    5877777777
    8877777777
    7777777777
    7777777777
    7777777777
    7777777777
    7777777777
    7777777777
    7777777777
    7777777777

After step 194:

    6988888888
    9988888888
    8888888888
    8888888888
    8888888888
    8888888888
    8888888888
    8888888888
    8888888888
    8888888888

After step 195:

    0000000000
    0000000000
    0000000000
    0000000000
    0000000000
    0000000000
    0000000000
    0000000000
    0000000000
    0000000000

If you can calculate the exact moments when the octopuses will all flash simultaneously, you should be able to navigate through the cavern. What is the first step during which all octopuses flash?

### Solution

Same as Part 1, just loop forever until we have a board of 0's and return which iteration it is.

In [213]:
def solve2(text):
    b, step = Board(text), 0
    while any([c for c in b.grid if c>0]):
        step += 1
        sim(b)
    return step

check(solve2(_sample2), 195)
check2(solve2(_input))

Part 2 Result: 329
